In [1]:
import os
import matplotlib.pyplot as plt
import ipywidgets as widgets
import pandas

from IPython.display import clear_output, update_display

/Users/rushikeshraut/work/stock/env/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [4]:
def get_data(sector_name):
    df = pandas.read_csv(f'History/{sector_name}_Data.csv')
    df = df.drop(columns=["Open", "High", "Low"])
    n = 4000
    df = df.head(n=n) # 2004-5
    df['Date'] = df['Date'].astype('datetime64')
    size = len(df['Close']) - 1
    first = df['Close'][n-1]
    df['Close'] = df['Close']*100/first
    df = df.rename(columns={"Close": sector_name})
    return df

list_of_files = os.listdir('History')
sector_options = [sector.split('_Data.')[0] for sector in list_of_files]
list_of_sectors = []
w = widgets.SelectMultiple(
    options=sector_options,
    value=list_of_sectors,
    description='Index',
    disabled=False
)
init_counter = 0
p_df = None
def on_dropdown_change(change):
    global p_df
    global init_counter
    global list_of_sectors
    if change['name'] == 'value' and (change['new'] != change['old']):
        list_of_sectors = change['new']
        
        for sector in list_of_sectors:
            if p_df is None:
                p_df = get_data(sector)
            if p_df is not None:
                if sector not in p_df.columns:
                    df = get_data(sector)
                    for column in df.columns:
                        if column != 'Date':
                            p_df[column] = df[column]
            for column in p_df.columns:
                if column not in list_of_sectors:
                    if column != 'Date':
                        p_df = p_df.drop(columns=[column])
                

        graph = p_df.plot(x='Date', y=[column for column in p_df.columns[1:]], figsize=(25,15))
                    
        clear_output()
        display(graph, display_id='graph')
        display(w, display_id='selectbox')
        

w.observe(on_dropdown_change)

display(w, display_id='selectbox')

SelectMultiple(description='Index', options=('NIFTYPrivateBank', 'NIFTYMidcap50', 'NIFTYMedia', 'NIFTYNext50',…

<DisplayHandle display_id=selectbox>

In [3]:
p_df

,Date,NIFTYBank,NIFTYFMCG,NIFTYIT,NIFTYNext50,NIFTY50
0,2021-01-28,1125.164930,1280.141088,119.226012,959.509230,749.385796
1,2021-01-27,1122.431545,1305.097974,121.883961,963.319119,757.518236
2,2021-01-25,1156.301425,1301.184362,122.628497,976.242050,772.237438
3,2021-01-22,1155.146917,1310.766619,124.826530,983.876584,779.450606
4,2021-01-21,1192.938046,1313.206915,124.639925,992.003954,791.298099
...,...,...,...,...,...,...
4195,2004-03-10,101.200095,97.652595,95.574954,101.114046,100.027117
4196,2004-03-09,104.086920,99.148751,95.755669,101.974296,101.204002
4197,2004-03-08,104.076172,100.071191,97.203982,102.625017,102.245302
4198,2004-03-05,101.186382,100.315602,99.358426,100.855823,101.293489
